# __1. 동네예보에서 각 항목별 데이터 스크래핑__  
### - 기온  
### - 체감온도  
### - 바람  
### - 습도  
### - 강수확률  
===> array or dataframe 구성  
## 1) 최고 / 최저 기온 출력  
## 2) 체감온도 평균 출력  
## 3) 바람이 가장 많이 부는 방향은?  
## 4) '맑음'에 해당되는 날씨가 가장 오랜 기간동안 지속되는 시각을 출력  
ex) 8월18일(화) 18시~8월19일(수) 24시

__import packages__

In [2]:
import urllib.request as ur
from bs4 import BeautifulSoup as bs
from selenium import webdriver as sw
import pandas as pd
import numpy as np
import time

__make soup obj__

In [3]:
url = 'https://www.weather.go.kr/weather/main.jsp'
driver = sw.Chrome('c:/mychrome/chromedriver.exe')
driver.get(url)
time.sleep(2)
html=driver.page_source
soup=bs(html,'html.parser')

__get & store from css selector__

In [4]:
weather_info=[]

# times
times = soup.select('div.tab01 > div.in-wid > ul:nth-child(3) > li')
times = [time.text for time in times]

# actual temperature
actual_temps = soup.select('g.highcharts-data-labels.highcharts-series-0.highcharts-line-series.highcharts-color-0 > g')
actual_temps = [temp.select('text>tspan')[1].text for temp in actual_temps]

# perceived temperature
perc_temps = soup.select('div.tab01 > div.in-wid > ul:nth-child(6) > li')
perc_temps = [temp.text[1:-1] for temp in perc_temps]

# wind
winds = soup.select('div.tab01 > div.in-wid > ul:nth-child(7) > li')
winds = [wind.select('p')[0].text for wind in winds]

# humidity
humid = soup.select('div.tab01 > div.in-wid > ul:nth-child(8) > li')
humid = [hu.select('p')[0].text for hu in humid]

# rain prob
rain_probs = soup.select('div.tab01 > div.in-wid > ul:nth-child(10) > li')
rain_probs = [rain.text for rain in rain_probs]
rain_probs.append(rain_probs[-1])

# weather
weathers = soup.select('div.tab01 > div.in-wid > ul.m-tb.wid83.pd67.bd-no.img01.clearfix > li')
weathers = [weather.select('img')[0]['title'] for weather in weathers]
weathers.append(weathers[-1])

# 풍향 
wind_dirs = soup.select('div.tab01 > div.in-wid > ul:nth-child(7) > li')
wind_dirs = [wind_dir.select('img')[0]['title'].strip() for wind_dir in wind_dirs]

# 날짜

days=[]
start=np.datetime64('today')
for i in range(len(times)):
    if times[i]=='3시':
        start+=1
    days.append(start)

__make dataframe__

In [5]:
df=pd.DataFrame()
df['날짜']=days
df['시간대']=times
df['날씨']=weathers
df['기온']=actual_temps
df['체감온도']=perc_temps
df['바람']=winds
df['풍향']=wind_dirs
df['습도']=humid
df['강수 확률']=rain_probs
df

,날짜,시간대,날씨,기온,체감온도,바람,풍향,습도,강수 확률
0,2020-08-19,12시,맑음,30℃,32℃,7km/h,남서풍,70%,0%
1,2020-08-19,15시,맑음,32℃,33℃,7km/h,남서풍,65%,0%
2,2020-08-19,18시,맑음,28℃,30℃,7km/h,서풍,70%,0%
3,2020-08-19,21시,맑음,26℃,29℃,4km/h,서풍,90%,0%
4,2020-08-19,24시,맑음,25℃,29℃,4km/h,남서풍,95%,0%
5,2020-08-20,3시,맑음,25℃,29℃,4km/h,남풍,95%,0%
6,2020-08-20,6시,맑음,24℃,27℃,4km/h,남풍,95%,0%
7,2020-08-20,9시,맑음,27℃,30℃,7km/h,남풍,90%,0%
8,2020-08-20,12시,맑음,31℃,33℃,7km/h,남서풍,75%,0%
9,2020-08-20,15시,맑음,32℃,33℃,7km/h,남서풍,65%,0%


__1.최고/최저 기온__

In [12]:
temp = np.array(list(map(lambda x:int(x[:-1]),df['기온'])))
th = np.max(temp)
tl = np.min(temp)

print(f'최고 기온 = {th}')
print(f'최저 기온 = {tl}')

최고 기온 = 32
최저 기온 = 24


__2.체감온도 평균__

In [13]:
temp = np.array(list(map(lambda x:int(x[:-1]),df['체감온도'])))
tav = np.mean(temp)

print(f'평균 체감온도 = {tav}')

평균 체감온도 = 30.095238095238095


__3.바람이 가장 많이 부는 방향__

In [14]:
wind_dict={}
for direction in df['풍향']:
    try:
        wind_dict[direction]+=1
    except KeyError :
        wind_dict[direction]=1

Ans,cnt='',0
for k,i in wind_dict.items():
    if i > cnt :
        Ans=k
        cnt=i
print(f'바람이 가장 많이 부는 방향 = {Ans}')

바람이 가장 많이 부는 방향 = 남풍


__4.'맑음'에 해당되는 날씨가 가장 오랜 기간동안 지속되는 시각__

In [32]:
s_day=''
cnt=0
e_day=''

tmp_s=''
tmp_cnt=0

for i in range(len(df['날씨'])):
    if df['날씨'][i] == '맑음':
        if tmp_cnt==0:
            tmp_s=[df['날짜'][i],df['시간대'][i]]
            tmp_cnt=1
        else:
            tmp_cnt+=1
    
    else:
        if tmp_cnt != 0:
            if tmp_cnt>cnt:
                s_day=tmp_s
                cnt=tmp_cnt
                e_day=[df['날짜'][i],df['시간대'][i]]
            tmp_s=''
            tmp_cnt=0

print(str(s_day[0])[5:10]+':'+s_day[1]+' ~ '+str(e_day[0])[5:10]+':'+e_day[1])

08-19:12시 ~ 08-21:9시
